In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pickle as pkl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Conv2DTranspose, Reshape, Input
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
# Dataset directories
train_data_dir = "Dataset/Preprocessed/Tomato/train_set"
test_data_dir = "Dataset/Preprocessed/Tomato/test_set"

In [ ]:
# Image dimensions and batch size
img_width, img_height = 400, 400
batch_size = 16

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode="categorical", shuffle=True)

test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode="categorical", shuffle=True)

#### Testing pretrained ResNet-50 model


In [ ]:
# Load the ResNet50 model
base_resNet_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_resNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(5, activation="softmax")(x)

# Create model
resnet_model = Model(inputs=base_resNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_resNet_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=0.001)
resnet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train model
history = resnet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=10, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save trained model
resnet_model.save("resNet_model.h5")

#### Testing DenseNet-121 model


In [ ]:
# Load pre-trained DenseNet121 model
base_denseNet_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Add custom classification head
x = base_denseNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(5, activation="softmax")(x)

# Create model
denseNet_model = Model(inputs=base_denseNet_model.input, outputs=predictions)

# Freeze convolutional layers
for layer in base_denseNet_model.layers:
    layer.trainable = False

# Compile model
optimizer = Adam(learning_rate=0.001)
denseNet_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train model
history = denseNet_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=10, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save trained model
denseNet_model.save("denseNet_model.h5")

#### Testing custom CNN model


In [ ]:
# Custom CNN architecture
custom_model = Sequential(
    [
        Conv2D(32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(5, activation="softmax"),
    ]
)

# Compile the model
custom_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
history = custom_model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=10, validation_data=test_generator, validation_steps=test_generator.samples // batch_size)

In [ ]:
# Save the trained model
custom_model.save("custom_CNN_model.h5")

#### Testing custom GAN model


In [ ]:
# # Generator
# def build_generator(latent_dim):
#     input_noise = Input(shape=(latent_dim,))
#     x = Dense(7 * 7 * 128, activation="relu")(input_noise)
#     x = Reshape((7, 7, 128))(x)
#     x = Conv2DTranspose(128, kernel_size=3, strides=2, padding="same", activation="relu")(x)
#     x = Conv2DTranspose(64, kernel_size=3, strides=2, padding="same", activation="relu")(x)
#     output_img = Conv2D(3, kernel_size=3, padding="same", activation="tanh")(x)
#     generator = Model(input_noise, output_img)
#     return generator

# latent_dim = 100
# generator = build_generator(latent_dim)


# # Discriminator
# def build_discriminator(input_shape):
#     input_img = Input(shape=input_shape)
#     x = Conv2D(64, kernel_size=3, strides=2, padding="same", activation="relu")(input_img)
#     x = Conv2D(128, kernel_size=3, strides=2, padding="same", activation="relu")(x)
#     x = Flatten()(x)
#     output = Dense(1, activation="sigmoid")(x)
#     discriminator = Model(input_img, output)
#     discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss="binary_crossentropy", metrics=["accuracy"])
#     return discriminator

# input_shape = (img_height, img_width, 3)  # Assuming 3 channels for RGB images
# discriminator = build_discriminator(input_shape)


# # Combined GAN model
# discriminator.trainable = False
# gan_input = Input(shape=(latent_dim,))
# gan_output = discriminator(generator(gan_input))
# gan = Model(gan_input, gan_output)
# gan.compile(optimizer=Adam(learning_rate=0.0002), loss="binary_crossentropy")

In [ ]:
# # Training GAN
# def train_gan(generator, discriminator, gan, train_generator, test_generator, latent_dim, epochs, batch_size):
#     num_batches = len(train_generator)
#     for epoch in range(epochs):
#         for batch in range(num_batches):
#             real_images, labels = next(train_generator)

#             noise = np.random.normal(0, 1, (batch_size, latent_dim))
#             fake_images = generator.predict(noise)

#             discriminator_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
#             discriminator_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
#             discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

#             noise = np.random.normal(0, 1, (batch_size, latent_dim))
#             gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

#             if batch % 100 == 0:
#                 print(f"Epoch: {epoch}, Batch: {batch}/{num_batches}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")

#     # Evaluate GAN performance on test set
#     test_loss, test_accuracy = gan.evaluate(test_generator)
#     print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# train_gan(generator, discriminator, gan, train_generator, test_generator, latent_dim, epochs=20, batch_size=batch_size)

In [ ]:
# # Save the trained generator model
# generator.save("gan_model.h5")